In [1]:
def takedata_index(csv , x):
    import random
    from tqdm import tqdm_notebook as tqdm
    
    plan = csv.plans.values
    click = csv.click_mode.values

    transport_mode = []
    index = []
    
    #100% means all data
    if(x == 100):
        csv.reset_index(drop=True, inplace=True) 
        return csv.index
    
    #put the most recommend plan in to transport_mode
    for i in range(len(plan)): 
        #it must have better way  but now i just want to quickly distingush the dif 
        #if mode = 10
        if(str(str(plan[i]).split('transport_mode')[1])[4] == str(0)):
            transport_mode.append(str(10))
        #if mode = 11
        elif(str(str(plan[i]).split('transport_mode')[1])[4] == str(1)):
            transport_mode.append(str(11))
        else:
            transport_mode.append(str(str(plan[i]).split('transport_mode')[1])[3])
    
        
    #find dif index between transport_mode and click_mode
    for i in range(len(click)): 
        if(int(transport_mode[i]) != click[i]):
            index.append(i)
    
    #put most recommend_mode people __% to training
    dif_id_len = len(index)
    normal_id_len = int(dif_id_len* (x/100))
    for i in tqdm(range(normal_id_len)):
        num = random.randint(1,len(plan))
        while(num in index):
            num = random.randint(1,len(plan))
        index.append(num)
    index.sort()
    
    return index

In [2]:
def train_data(file, index):
    import pandas as pd
    from sklearn import preprocessing
    from sklearn.preprocessing import MinMaxScaler
    
    train_d = file.iloc[index]
    train_label = train_d.click_mode
    #choose what feature you don't need
    train_d = train_d.drop(columns= ['sid','pid','plans','req_time','plan_time','o','d','click_mode','click_time'])
    train_d = train_d.drop(columns = ['time_hour'])
    train_d.reset_index(drop=True, inplace=True)
    
    #normalize
    scaler = MinMaxScaler()
    train_d = scaler.fit_transform(train_d)
    train_d = pd.DataFrame(train_d)
    
    return train_d, train_label

In [3]:
def rf(data, label):
    from sklearn import preprocessing, cross_validation, metrics, ensemble
    
    train_X, test_X, train_y, test_y = cross_validation.train_test_split(data, label, test_size = 0.3)
    forest = ensemble.RandomForestClassifier(n_estimators = 100)
    forest_fit = forest.fit(train_X, train_y)
    test_y_predicted = forest.predict(test_X)

    accuracy = metrics.accuracy_score(test_y, test_y_predicted)
    print(forest_fit)
    print(accuracy)

In [4]:
def main():
    import pandas as pd
    import numpy as np
    import csv
#     from sklearn import preprocessing, cross_validation, metrics, ensemble
#     from sklearn.preprocessing import MinMaxScaler
    
    train_final = pd.read_csv('/storage/kdd/train_final2.csv')
    
    index = takedata_index(train_final ,100)
    train_d, train_label = train_data(train_final, index)
    rf(train_d, train_label)

In [5]:
if __name__ == '__main__':
    main()

/usr/local/lib/python3.5/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)
0.6883919971176683
